In [6]:
import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
haar_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
without_mask=np.load('DataWithoutMask.npy')
with_mask=np.load('DataWithMask.npy')
with_mask = with_mask.reshape(400,50*50*3)
without_mask = without_mask.reshape(400,50*50*3)
x = np.r_[with_mask,without_mask]
lables = np.zeros(x.shape[0])
lables[400:]=1.0
x_train , x_test , y_train , y_test = train_test_split(x,lables,test_size=0.30)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)
svm = SVC()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test,y_pred))


0.9916666666666667


In [ ]:
import tkinter as tk
from tkinter import filedialog
def camera(root):
    root.destroy()
    import time
    capture = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_COMPLEX
    while True:
        flag,img=capture.read()
        if flag:
            face = haar_data.detectMultiScale(img)
            for x,y,w,h in face:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),3)
                face = img[y : y+h , x : w+x ,:]
                face = cv2.resize(face,(50,50))
                face = face.reshape(1,-1)
                face = pca.transform(face)
                what = svm.predict(face)
                time.sleep(0.1)
                if what:
                    n = "without mask"
                else:
                    n = "with mask"
                cv2.putText(img,n,(x,y),font,1,(255,0,255),1)
            cv2.imshow('result',img)
            if cv2.waitKey(2)==27 : #ascii of escape key
                break
    capture.release()
    cv2.destroyAllWindows()
    create_menu()
def mobile(root):
    root.destroy()
    import time
    import urllib.request
    url="http://192.168.1.5:8080/shot.jpg"
    font = cv2.FONT_HERSHEY_COMPLEX
    while True:
        img_arr=np.array(bytearray(urllib.request.urlopen(url).read()),dtype=np.uint8)
        img=cv2.imdecode(img_arr,-1)
        face = haar_data.detectMultiScale(img)
        for x,y,w,h in face:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),3)
            face = img[y : y+h , x : w+x ,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            what = svm.predict(face)
            time.sleep(0.1)
            if what:
                n = "without mask"
            else:
                n = "with mask"
            cv2.putText(img,n,(x,y),font,1,(255,0,255),1)
            cv2.imshow('result',img)
            if cv2.waitKey(2)==27 : #ascii of escape key
                break
    capture.release()
    cv2.destroyAllWindows()
    create_menu()
def selectimage(root):
    root.destroy()
    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg")])
    img = cv2.imread(file_path)
    font = cv2.FONT_HERSHEY_COMPLEX
    while True:
        face = haar_data.detectMultiScale(img)
        for x,y,w,h in face:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),3)
            face = img[y : y+h , x : w+x ,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            what = svm.predict(face)
            if what:
                n = "without mask"
            else:
                n = "with mask"
            cv2.putText(img,n,(x,y),font,1,(255,0,255),1)
        cv2.imshow('result',img)
        if cv2.waitKey(2)==27: #ascii of escape key
            break
    cv2.destroyAllWindows()
    create_menu()
def Exit(root):
    root.destroy()
def create_menu():
    root = tk.Tk()
    root.title("GUI Menu")

    window_width = 400
    window_height = 300

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    x_coordinate = int((screen_width / 2) - (window_width / 2))
    y_coordinate = int((screen_height / 2) - (window_height / 2))

    root.geometry(f"{window_width}x{window_height}+{x_coordinate}+{y_coordinate}")

    menu_frame = tk.Frame(root, pady=20)
    menu_frame.pack()

    button1 = tk.Button(menu_frame, text="Check with Laptop", width=20, command=lambda:camera(root))
    button1.pack(pady=10)

    button2 = tk.Button(menu_frame, text="Check with Mobile", width=20, command=lambda:mobile(root))
    button2.pack(pady=10)

    button3 = tk.Button(menu_frame, text="Check on Selected Image", width=20, command=lambda:selectimage(root))
    button3.pack(pady=10)
    
    button4 = tk.Button(menu_frame, text="Exit", width=20, command=lambda:Exit(root))
    button4.pack(pady=10)

    root.mainloop()


create_menu()

In [9]:
def selectimage():
    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg")])
    img = cv2.imread(file_path)
    font = cv2.FONT_HERSHEY_COMPLEX
    while True:
        face = haar_data.detectMultiScale(img)
        for x,y,w,h in face:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),4)
            face = img[y : y+h , x : w+x ,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            what = svm.predict(face)
            if what:
                n = "without mask"
            else:
                n = "with mask"
            cv2.putText(img,n,(x,y),font,1,(255,0,255),2)
        cv2.imshow('result',img)
        if cv2.waitKey(2)==27: #ascii of escape key
            break
    cv2.destroyAllWindows()

In [ ]:
def mobile():
    import time
    import urllib.request
    url="http://192.168.1.5:8080/shot.jpg"
    font = cv2.FONT_HERSHEY_COMPLEX
    while True:
        img_arr=np.array(bytearray(urllib.request.urlopen(url).read()),dtype=np.uint8)
        img=cv2.imdecode(img_arr,-1)
        face = haar_data.detectMultiScale(img)
        for x,y,w,h in face:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),4)
            face = img[y : y+h , x : w+x ,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            #face = pca.transform(face)
            what = svm.predict(face)
            time.sleep(0.1)
            if what:
                n = "without mask"
            else:
                n = "with mask"
            cv2.putText(img,n,(x,y),font,1,(255,0,255),2)
            cv2.imshow('result',img)
            if cv2.waitKey(2)==27 : #ascii of escape key
                break
    capture.release()
    cv2.destroyAllWindows()